### **Train and Test the Models on the LISA Dataset**

Three models for traffic sign classification:
1. CNNsmall (original LISA-CNN)
2. CNNlarge (original GTSRB-CNN)
3. CNN-STN 

Five generic image classification models
1. ResNet18
2. EfficientNet-B0
3. DenseNet-121
4. MobileNetv2
5. ShuffleNetv2

In [1]:
import pickle

with open('dataset/LISA/train.pkl', 'rb') as f:
    train = pickle.load(f)
    train_data, train_labels = train['data'], train['labels']
with open('dataset/LISA/test.pkl', 'rb') as f:
    test = pickle.load(f)
    test_data, test_labels = test['data'], test['labels']

# Calculate the sizes in percentage
total_size = len(train_data) + len(test_data)
train_percentage = (len(train_data) / total_size) * 100
test_percentage = (len(test_data) / total_size) * 100

print(f"Training data size: {len(train_data)} which is {train_percentage:.2f}% of the total dataset")
print(f"Test data size: {len(test_data)} which is {test_percentage:.2f}% of the total dataset")

Training data size: 5467 which is 80.00% of the total dataset
Test data size: 1367 which is 20.00% of the total dataset


In [2]:
import pickle
import os
import json
from models import *
from utils import *
from torchvision.models import mobilenet_v2, efficientnet_b0, densenet121, shufflenet_v2_x1_0

with open('classes.json', 'r') as config:
    params = json.load(config)
    class_n = params['LISA']['class_n']
    device = params['device']
    labels = params['GTSRB']['labels']

In [3]:
def train_model(model_name, adv_train=False):

    with open('dataset/LISA/train.pkl', 'rb') as f:
        train = pickle.load(f)
        train_data, train_labels = train['data'], train['labels']
    with open('dataset/LISA/test.pkl', 'rb') as f:
        test = pickle.load(f)
        test_data, test_labels = test['data'], test['labels']

    if model_name=='TransformerLISA':
        training_model = Transformer(class_n=class_n).to(device).apply(weights_init)
    elif model_name=='CNNsmallLISA':
        training_model = CNNsmall(class_n=class_n).to(device).apply(weights_init)
    elif model_name=='CNNlargeLISA':
        training_model = CNNlarge(class_n=class_n).to(device).apply(weights_init)
    elif model_name=='ResNet18LISA':
        training_model = ResNet18(class_n=class_n).to(device) #.apply(weights_init)
    elif model_name=='MobileNetv2LISA':
        training_model = mobilenet_v2(num_classes=class_n).to(device) #.apply(weights_init)
    elif model_name=='EfficientNetLISA':
        training_model = efficientnet_b0(num_classes=class_n).to(device) #.apply(weights_init)
    elif model_name=='DenseNet121LISA':
        training_model = densenet121(num_classes=class_n).to(device) #.apply(weights_init)
    elif model_name=='ShuffleNetv2x1LISA':
        training_model = shufflenet_v2_x1_0(num_classes=class_n).to(device) #.apply(weights_init)


    training(training_model=training_model,
             train_data=train_data,
             train_labels=train_labels,
             test_data=test_data,
             test_labels=test_labels,
             model_name=model_name,
             device=device)


def test_model(model_name):

    if model_name=='TransformerLISA':
        trained_model = Transformer(class_n=class_n).to(device)
    elif model_name=='CNNsmallLISA':
        trained_model = CNNsmall(class_n=class_n).to(device)
    elif model_name=='CNNlargeLISA':
        trained_model = CNNlarge(class_n=class_n).to(device)
    elif model_name=='ResNet18LISA':
        trained_model = ResNet18(class_n=class_n).to(device)
    elif model_name=='MobileNetv2LISA':
        trained_model = mobilenet_v2(num_classes=class_n).to(device)
    elif model_name=='EfficientNetLISA':
        trained_model = efficientnet_b0(num_classes=class_n).to(device)
    elif model_name=='DenseNet121LISA':
        trained_model = densenet121(num_classes=class_n).to(device)
    elif model_name=='ShuffleNetv2x1LISA':
        trained_model = shufflenet_v2_x1_0(num_classes=class_n).to(device)

    trained_model.load_state_dict(
        torch.load(f'models/{model_name}.pth',
                   map_location=torch.device(device)))

    with open('dataset/LISA/test.pkl', 'rb') as f:
        test = pickle.load(f)
        test_data, test_labels = test['data'], test['labels']

    test_set = TrafficSignDataset(test_data, test_labels, device)
    test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

    trained_model.eval()
    with torch.no_grad():
        test_acc, _, inference_time = model_epoch(trained_model, test_loader)

    # Calculate average inference time per sample
    avg_inference_time = inference_time / len(test_set)

    print(f'Test Acc: {round(float(test_acc / test_set.__len__()), 4)}')
    print(f'Average Inference Time: {avg_inference_time * 1000.0:.5f} ms')

##### **Training**

In [4]:
# train_model(model_name='DenseNet121LISA')
# train_model(model_name='ShuffleNetv2x1LISA')

##### Testing**

In [5]:
test_model(model_name='TransformerLISA')

Test Acc: 0.9985
Average Inference Time: 0.01426 ms


/fzi/ids/pavlitsk/.local/lib/python3.8/site-packages/torch/nn/functional.py:4255: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/fzi/ids/pavlitsk/.local/lib/python3.8/site-packages/torch/nn/functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [6]:
test_model(model_name='CNNsmallLISA')

Test Acc: 0.9971
Average Inference Time: 0.00705 ms


In [7]:
test_model(model_name='CNNlargeLISA')

Test Acc: 0.9978
Average Inference Time: 0.01114 ms


In [8]:
test_model(model_name='ResNet18LISA')

Test Acc: 0.9985
Average Inference Time: 0.02994 ms


In [9]:
test_model(model_name='MobileNetv2LISA')

Test Acc: 0.9971
Average Inference Time: 0.05163 ms


In [10]:
test_model(model_name='EfficientNetLISA')

Test Acc: 0.9934
Average Inference Time: 0.09199 ms


In [11]:
test_model(model_name='DenseNet121LISA')

Test Acc: 0.9963
Average Inference Time: 0.19866 ms


In [12]:
test_model(model_name='ShuffleNetv2x1LISA')

Test Acc: 0.9927
Average Inference Time: 0.09243 ms


In [13]:
from torchinfo import summary

model = shufflenet_v2_x1_0(class_n).to(device)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

2278604
